In [ ]:
## Setup
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset,DataLoader
from datetime import datetime
import p3DNets


In [ ]:
"""
#import torch.nn.modules.conv as conv
#import CoordConv
#from tensorflow.python.keras.layers import Layer, InputSpec
#from tensorflow.keras import backend as K
#from tensorflow.keras.utils import get_custom_objects
"""

'\n#import torch.nn.modules.conv as conv\n#import CoordConv\n#from tensorflow.python.keras.layers import Layer, InputSpec\n#from tensorflow.keras import backend as K\n#from tensorflow.keras.utils import get_custom_objects\n'

In [ ]:
def try_gpu():
    """
    If GPU is available, return torch.device as cuda:0; else return torch.device
    as cpu.
    """
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu')
    return device
# Try using gpu instead of cpu
device = try_gpu()

In [ ]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
      if hasattr(layer, 'reset_parameters'):
          print(f'Reset trainable parameters of layer = {layer}')
          layer.reset_parameters()
    
def custom_loss(y_pred, y_batch):
    
    loss = torch.mean(torch.abs(y_batch - y_pred))*128
    
    constraint = torch.sum(torch.abs(-4 - torch.sum(y_pred, axis = 1)))/(y_batch.shape[0])
    
    return (loss + constraint)

def custom_loss_orig(y_pred, y_batch):
    
    loss = torch.mean(torch.abs(y_batch - y_pred))
    
    constraint = torch.sum(torch.abs(-4 - torch.sum(y_pred, axis = 1)))/(128*(y_batch.shape[0]))
    
    return (loss + constraint)

def evaluate_accuracy(data_loader, net, device):
    """Evaluate accuracy of a model on the given data set."""
    net.eval()  #make sure network is in evaluation mode

    #init
    loss_sum = torch.tensor([0], dtype=torch.float32, device=device)
    con_sum = torch.tensor([0], dtype=torch.float32, device=device)
    n = 0

    for X, y in data_loader:
        # Copy the data to device.
        X, y = X.to(device).to_dense(), y.to(device)
        with torch.no_grad():
         
            loss_sum += torch.sum(torch.abs(net(X) - y))/128
            con_sum += torch.sum(torch.abs(torch.sum(net(X), axis = 1) - torch.sum(y,axis = 1)))/128
            
            n += y.shape[0] #increases with the number of samples in the batch
    return loss_sum.item()/n, con_sum.item()/n


def train_test(epochs, net, device, train_loader, test_loader, optimizer, criterion):
    """Training and testing function of FCN"""
    
    startTime = datetime.now()
    
    # Define list to store losses and performances of each iteration
    train_losses = []
    train_losss = []
    test_losss = []
    train_cons = []
    test_cons = []
    
    for epoch in range(epochs):

        # Network in training mode and to device
        net.train()
        net.to(device)
        
        #for name, module in net.named_modules():
        #    if isinstance(module, torch.nn.Conv2d):
        #        module.weight = torch.nn.Parameter(module.weight.data.to_sparse())
        #        module.bias = torch.nn.Parameter(module.bias.data.to_sparse())

        # Training loop
        for i, (x_batch, y_batch) in enumerate(train_loader):

            # Set to same device
            
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            x_batch = x_batch.to_dense()
            
            # Set the gradients to zero
            optimizer.zero_grad()

            # Perform forward pass
            y_pred = net(x_batch)

            # Compute the loss
            loss = criterion(y_pred, y_batch)
            train_losses.append(loss.detach().cpu().numpy())
            
            # Backward computation and update
            loss.backward()
            optimizer.step()
            
            #print(psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2)
           
        print(datetime.now() - startTime)
        
        elif epoch == 25: 
            learning_rate = 0.0001
            optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=0.5)
        elif epoch == 50:
            learning_rate = 0.00001
            optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=0.5)
        elif epoch == 75:
            learning_rate = 0.000001
            optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=0.5) 
                    
        if  epoch == 0 or epoch%5 == 0:
            # Compute train and test error
            train_loss, train_con = evaluate_accuracy(train_loader, net.to(device), device)
            test_loss, test_con = evaluate_accuracy(test_loader, net.to(device), device)
            
            # Development of performance
            train_losss.append(train_loss)
            test_losss.append(test_loss)
            
            train_cons.append(train_con)
            test_cons.append(test_con)
    
            # Print performance
            print('Epoch: {:.0f}'.format(epoch+1))
            print('Loss on train set: {:.7f}'.format(train_loss))
            print('Loss on test set: {:.7f}'.format(test_loss))
            print('Con on train set: {:.7f}'.format(train_con))
            print('Con on test set: {:.7f}'.format(test_con))
            print('')

    print(datetime.now() - startTime)
    
    return net, train_losses, train_losss, test_losss, train_cons, test_cons


In [ ]:
in_channels = 1
hidden_channels = [4,8,16,32,64,128,256]
out_features = 128

learning_rate = 0.001
epochs = 101
criterion = custom_loss
criterion_orig = custom_loss_orig

In [ ]:
test_loader = torch.load('test_loader_d_3D')
train_loader = torch.load('train_loader_d_3D_8')

In [ ]:
test_loader_H2O = torch.load('test_loader_H2O_3D')
train_loader_H2O = torch.load('train_loader_H2O_3D')

hidden_channels_res = [16,32,64,128]
net_res_H2O = p3DNets.ResNet18_3D_d(in_channels, hidden_channels_res)
optimizer_res_H2O = torch.optim.Adam(net_res_H2O.parameters(), lr=learning_rate, weight_decay=0.5)
net_res_H2O.apply(reset_weights)
net_res_H2O_res_H2O, train_losses_res_H2O, train_losss_res_H2O, test_losss_res_H2O, train_cons_res_H2O, test_cons_res_H2O = train_test(epochs, net_res_H2O, device, train_loader_H2O, test_loader_H2O, optimizer_res_H2O, criterion)
net_res_H2O_res_H2O = net_res_H2O_res_H2O.cpu()
torch.save(net_res_H2O_res_H2O,'drive/MyDrive/Thesis/net_res_H2O_res_H2O')
np.savez('drive/MyDrive/Thesis/net_res_H2O_3D.npz', trainlossd=train_losses_res_H2O, trainlosssd=train_losss_res_H2O, testlosssd=test_losss_res_H2O, traincond=train_cons_res_H2O, testcond=test_cons_res_H2O)

test_loader_xyz = torch.load('test_loader_xyz_3D')
train_loader_xyz = torch.load('train_loader_xyz_3D')


Reset trainable parameters of layer = Conv3d(1, 16, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3))
Reset trainable parameters of layer = BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Reset trainable parameters of layer = Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
Reset trainable parameters of layer = BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Reset trainable parameters of layer = BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Reset trainable parameters of layer = Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
Reset trainable parameters of layer = Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
Reset trainable parameters of layer = BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Reset trainable parameters of layer = BatchNorm3d(16, eps=1e-05, momentum=0.1

KeyboardInterrupt: ignored